<a href="https://colab.research.google.com/github/PedroPepeu/CUDA-pratical-class/blob/main/CUDA_Pratical_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Python 3.11.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpsjj68za0".


In [ ]:
# Verify GPU access
!nvidia-smi

Wed May 21 14:45:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [13]:
%%writefile vector_ops.cu
#include <stdio.h>  // Para funções de entrada/saída (printf)
#include <stdlib.h> // Para funções de entrada/saída (printf)
#include <math.h>   // Para funções matemáticas

// CHECAGEM DE ERRO, CHAMA FUNÇAO VER SE DA ERRO MOSTRA LOG DE ERRO
#define CHECK_CUDA_ERROR(call) \
do { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA Error: %s:%d, %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
        exit(EXIT_FAILURE); \
    } \
} while(0)

// Kernel for vector addition
__global__ void vectorAdd(float *a, float *b, float *result, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        result[i] = a[i] + b[i];
    }
}

// Kernel for vector subtraction
__global__ void vectorSub(float *a, float *b, float *result, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        result[i] = a[i] - b[i];
    }
}

// Kernel for vector multiplication
__global__ void vectorMul(float *a, float *b, float *result, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        result[i] = a[i] * b[i];
    }
}

// Kernel for vector division
__global__ void vectorDiv(float *a, float *b, float *result, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        if (b[i] != 0) {  // Avoid division by zero
            result[i] = a[i] / b[i];
        } else {
            result[i] = 0;  // Or handle differently as needed
        }
    }
}

//Esta função imprime apenas os primeiros 5 elementos do vetor para não poluir a saída.
void printVector(float *vec, int n, const char* name) {
    printf("%s: [", name);
    for (int i = 0; i < (n < 5 ? n : 5); i++) {
        printf("%.2f", vec[i]);
        if (i < (n < 5 ? n - 1 : 4)) printf(", ");
    }
    if (n > 5) printf(", ...");
    printf("]\n");
}

int main() {
    int n = 10; // Tamanho do vetor
    size_t size = n * sizeof(float); // Tamanho em bytes

    // Host vectors
    float *h_a = (float *)malloc(size); // Vetor A
    float *h_b = (float *)malloc(size); // Vetor B
    float *h_result = (float *)malloc(size); // Vetor resultado

    if (h_a == NULL || h_b == NULL || h_result == NULL) {
        printf("Host memory allocation failed\n");
        return -1;
    }

    // Initialize vectors
    for (int i = 0; i < n; i++) {
        h_a[i] = i + 1.0;       // [1, 2, 3, ...]
        h_b[i] = (i + 1) * 2.0;  // [2, 4, 6, ...]
    }

    // Alocação de Memória na GPU: d_ prefixo indica memória na GPU (device)
    float *d_a, *d_b, *d_result;
    CHECK_CUDA_ERROR(cudaMalloc(&d_a, size));
    CHECK_CUDA_ERROR(cudaMalloc(&d_b, size));
    CHECK_CUDA_ERROR(cudaMalloc(&d_result, size));

    //  Cópia de Dados para a GPU
    CHECK_CUDA_ERROR(cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice));
    CHECK_CUDA_ERROR(cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice));

    // Configuração de Execução
    int threadsPerBlock = 256; // Threads por bloco
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;// Número de blocos

    // Perform each operation and print results
    printf("Vector Operations:\n");
    printVector(h_a, n, "A");
    printVector(h_b, n, "B");

    // Addition
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_result, n);
    CHECK_CUDA_ERROR(cudaGetLastError());
    CHECK_CUDA_ERROR(cudaDeviceSynchronize());
    CHECK_CUDA_ERROR(cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost));
    printVector(h_result, n, "A + B");

    // Subtraction
    vectorSub<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_result, n);
    CHECK_CUDA_ERROR(cudaGetLastError());
    CHECK_CUDA_ERROR(cudaDeviceSynchronize());
    CHECK_CUDA_ERROR(cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost));
    printVector(h_result, n, "A - B");

    // Multiplication
    vectorMul<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_result, n);
    CHECK_CUDA_ERROR(cudaGetLastError());
    CHECK_CUDA_ERROR(cudaDeviceSynchronize());
    CHECK_CUDA_ERROR(cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost));
    printVector(h_result, n, "A * B");

    // Division
    vectorDiv<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_result, n);
    CHECK_CUDA_ERROR(cudaGetLastError());
    CHECK_CUDA_ERROR(cudaDeviceSynchronize());
    CHECK_CUDA_ERROR(cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost));
    printVector(h_result, n, "A / B");

    // Cleanup:Libera toda a memória alocada, tanto na CPU quanto na GPU.
    free(h_a);
    free(h_b);
    free(h_result);
    CHECK_CUDA_ERROR(cudaFree(d_a));
    CHECK_CUDA_ERROR(cudaFree(d_b));
    CHECK_CUDA_ERROR(cudaFree(d_result));

    return 0;
}

Overwriting vector_ops.cu


In [14]:
!nvcc vector_ops.cu -o vector_ops -arch=sm_75
!./vector_ops

Vector Operations:
A: [1.00, 2.00, 3.00, 4.00, 5.00, ...]
B: [2.00, 4.00, 6.00, 8.00, 10.00, ...]
A + B: [3.00, 6.00, 9.00, 12.00, 15.00, ...]
A - B: [-1.00, -2.00, -3.00, -4.00, -5.00, ...]
A * B: [2.00, 8.00, 18.00, 32.00, 50.00, ...]
A / B: [0.50, 0.50, 0.50, 0.50, 0.50, ...]


In [8]:
%%writefile custom_formula.cu
#include <stdio.h>
#include <stdlib.h>

/*
 * STUDENT TASK: Implement your custom formula here
 *
 * The formula should use elements from both vectors a and b
 * Example formulas you could implement:
 * 1. (a + b) / 2         (Average)
 * 2. a² + b²             (Sum of squares)
 * 3. sqrt(a² + b²)       (Magnitude)
 * 4. a * b + a + b       (Custom polynomial)
 *
 * Choose one or create your own!
 */
__global__ void customFormula(float *a, float *b, float *result, int n) {
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    if (i < n) {
        // STUDENT TODO: Implement your formula here
        // Example: result[i] = (a[i] + b[i]) / 2.0f;
        result[i] = 0.0f; // Change this!
    }
}

void printVector(float *vec, int n) {
    printf("[");
    for (int i = 0; i < min(n, 5); i++) {
        printf("%.2f", vec[i]);
        if (i < min(n, 5) - 1) printf(", ");
    }
    if (n > 5) printf(", ...");
    printf("]\n");
}

int main() {
    int n = 10; // Vector size
    size_t size = n * sizeof(float);

    // Host vectors
    float *h_a = (float *)malloc(size);
    float *h_b = (float *)malloc(size);
    float *h_result = (float *)malloc(size);

    // Initialize vectors
    for (int i = 0; i < n; i++) {
        h_a[i] = i + 1.0f;       // [1, 2, 3, ...]
        h_b[i] = (i + 1) * 2.0f;  // [2, 4, 6, ...]
    }

    // Device vectors
    float *d_a, *d_b, *d_result;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_result, size);

    // Copy to device
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

    // Launch configuration
    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;

    // Run custom formula
    customFormula<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_result, n);
    cudaMemcpy(h_result, d_result, size, cudaMemcpyDeviceToHost);

    // Print results
    printf("Vector A: ");
    printVector(h_a, n);
    printf("Vector B: ");
    printVector(h_b, n);
    printf("Custom formula result: ");
    printVector(h_result, n);

    // Cleanup
    free(h_a);
    free(h_b);
    free(h_result);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_result);

    return 0;
}

Writing custom_formula.cu


In [ ]:
!nvcc custom_formula.cu -o custom_formula
!./custom_formula